In [18]:
#>1#(33080024):(33080024)
import os
#<1#(33080024)~%(-208587713) #>1#(65545957):(65545957)
import boto3
#<1#(65545957)~%(769109360)
# Leave for imports

ModuleNotFoundError: No module named 'boto3'

Run the following !pip3 cells to install boto3 which is needed for this sandbox.

In [ ]:
!pip3 install boto3

The big take away here? When we script, its so much easier to through in an automation that'll do what we need rather that try to wire up libraries, search stack overflow, etc. 

# Automations and Scripting


A common use of python, and other languages, is to write quick scripts. But, it doesn't long for quick scripts to become permenent, or, quick scripts to become lengthy scripts. Worst of all, a "quick script" can too easily become an integral part of a legacy software package. 

Automations are beneficial for scripting as well! Whether reducing common expressions or interfacing with APIs, automations can make it simpler. 

In this sandbox, we'll use managing AWS EC2 virtual machines as an example. What do automations let you focus on? What you want to - creating, connecting, listing, terminating, etc. 

##### Note
This sandbox does require an AWS account to truly experience. Please have your AWS keys ready, they will remain private (unless you publish them to a public repo... don't do that...)

### Setup

First, we need to pass in our AWS keys. Typically, you would do this in a setup file, like Setup.py, but we've placed them here for ease. Clearly, the keys below are placeholders. Put in your AWS account keys to fully experience the sandbox. 

In [ ]:
#[AWSAccessKey 0123456789abcdefghij]
#[AWSSecretAccessKey 0123456789a/0123456/0123456789abcdefghij]

The next automation isn't essential, but we have it here for simplicity. If it was missing, the first AWS automation would create the client. 

In [ ]:
#[AWS.Client]#@65545957 #>1#(65545957):(65545957)
ec2_client = boto3.client('ec2', region_name='us-west-2',
    aws_access_key_id='0123456789abcdefghij' ,
    aws_secret_access_key='0123456789a/0123456/0123456789abcdefghij')
#<1#(65545957)~%(-38374046)

### Create Instance
First, let's create an AWS VM instance. Now, typically this requires jump through some hoops, but we'll automate the code to jump through those hoops for you. The automation will create code to save the .pem key and manage it for you. It will also save information within Arctic Fox concerning the instance's operating system. Please be aware, more code may be generated than you'd expect. 

Creating an instance does require a name, we pass it in as the first item. 

In [ ]:
#[AWS.Instance codespaceExample --create]#@15180284 #>1#(15180284):(15180284)
#***First, create a key for the new instances
#[AWS.Keys --list --names]#@65000917
existingKeys : any #<1#(15180284)~%(-916180027) #>2#(65000917,15180284):(65000917)
existingKeys = ec2_client.describe_key_pairs()
existingKeys = [key['KeyName'] for key in existingKeys['KeyPairs']]
#<2#(65000917)~%(73998777) #>1#(15180284):(15180284)
if 'codespaceExampleKey' not in existingKeys:
    #[AWS.Keys --create --name codespaceExampleKey]#@39183305 #<1#(15180284)~%(-1064226032)
#>2#(39183305,15180284):(39183305)
    key_pair = ec2_client.create_key_pair(KeyName="codespaceExampleKey")
    private_key = key_pair["KeyMaterial"]

    # write private key to file with 400 permissions
    with os.fdopen(os.open("codespaceExampleKey.pem", os.O_WRONLY | os.O_CREAT, 0o400), "w+") as handle:
        handle.write(private_key)
        print('Key named: codespaceExampleKey, saved to file codespaceExampleKey.pem')
    with os.fdopen(os.open("/home/codespace/.icii/arcticfox/codespaceExampleKey.pem", os.O_WRONLY | os.O_CREAT, 0o400), "w+") as handle:
        handle.write(private_key)
        print('Also saved to file /home/codespace/.icii/arcticfox/codespaceExampleKey.pem')
#<2#(39183305)~%(-943315609) #>1#(15180284):(15180284)
else:
    print('Key named: codespaceExampleKey, already exists')
#<2#(17386517,15180284)~%(-921090987)

In [ ]:
#>2#(17386517,15180284):(17386517)
#***Generated by: [AWS.Instance --list --names --states]
def getAllInstanceNamesAndStates():
    instanceNames = []
    instanceStates = []
    #[AWS.SearchInstancesFor]#@85920194 #<2#(15180284)~%(-2120946417)
#>3#(85920194,17386517,15180284):(85920194)
    allInstances = ec2_client.describe_instances()
    for reservation in allInstances['Reservations']:
        for possibleInstance in reservation['Instances']:
#<3#(85920194)~%(1421253349) #>2#(17386517,15180284):(17386517)
            for tag in possibleInstance['Tags']:
                    if tag['Key'] == 'Name':
                        instanceNames.append(tag['Value'])
                        instanceStates.append((tag['Value'], possibleInstance['State']['Name']))
    return (instanceNames, instanceStates)
print('Function: getAllInstanceNamesAndStates, compiled')
#<2#(17386517,15180284)~%(-183528672)

In [ ]:
#>1#(15180284):(15180284)
#***Generated by: [AWS.Instance codespaceExample --create]
#[AWS.Instance --list --names --states]#@17386517
existingInstanceTuple : any  #<1#(15180284)~%(-2036225202) #>2#(17386517,15180284):(17386517)
existingInstanceTuple = getAllInstanceNamesAndStates()
#<2#(17386517)~%(1243525709) #>1#(15180284):(15180284)
existingInstanceNames = existingInstanceTuple[0]
existingInstanceStates = existingInstanceTuple[1]

if "codespaceExample" not in existingInstanceNames or [nameState for nameState in existingInstanceStates if nameState[0] == "codespaceExample"][0] == 'terminated':
    #***Second, create the new instance(s)
    instances = ec2_client.run_instances(
        ImageId="ami-0c79a55dda52434da",
        MinCount=1,
        MaxCount=1,
        InstanceType='t4g.micro',
        KeyName='codespaceExampleKey',
        TagSpecifications=[
            {
                'ResourceType': 'instance',
                'Tags': [
                    {
                        'Key': 'Name',
                        'Value': 'codespaceExample'
                    },
                ]
            },
        ],
    )

    print('Instance: codespaceExample, created and starting up!')

else:
    print('The instance name: codespaceExample, is already used')
#<1#(55166203,33080024)~%(834886808)

In [ ]:
#>1#(55166203,33080024):(55166203,33080024)
#***Generated by: [AWS.InstanceByName codespaceExample]
def findInstanceByName(name: str):
    #[AWS.SearchInstancesFor]#@64602593 #<2#(55166203,33080024)~%(-1447125068)
#>3#(64602593,55166203,33080024):(64602593)
    allInstances = ec2_client.describe_instances()
    for reservation in allInstances['Reservations']:
        for possibleInstance in reservation['Instances']:
#<3#(64602593)~%(1421253349) #>2#(55166203,33080024):(55166203,33080024)
            for tag in possibleInstance['Tags']:
                if tag['Key'] == 'Name' and tag['Value'] == name:
                    return possibleInstance
#<2#(55166203,33080024)~%(-1031880091)

### Connect to the Instance

Using the same AWS.Instance automation, let's pass in the item --connect. This time, the automation will generate code to connect to the named instance. Also, you don't have to find the .pem, format it correctly, find the ip address, or discern the correct username. No, automations do all that for you. 

In [ ]:
#[AWS.Instance codespaceExample --connect]#@33080024 #>1#(33080024):(33080024)
#[AWS.InstanceByName codespaceExample]#@55166203
instance : any #<1#(33080024)~%(-1766556229) #>2#(55166203,33080024):(55166203)
instance = findInstanceByName("codespaceExample")
#<2#(55166203)~%(30964080) #>1#(33080024):(33080024)
connectCommand = "ssh -i \"/home/codespace/.icii/arcticfox/" + instance['KeyName'] + ".pem\" ubuntu@" + instance['PublicDnsName']
print("Using a codespace, can't launch a new window...")
print("Copy and paste the following command into an integrated terminal:")
print(connectCommand)
#<1#(33080024)~%(-31368841)

### List Instances
Other things we want to do when scripting is see what resources are in use. We added the ability to list instances so we could easily see what VMs were running. 

Just listing the instances gives a bit too much information to digest...

In [ ]:
#[AWS.Instance --list]#@14984901 #>1#(14984901):(14984901)
allInstances = [reservation['Instances'] for reservation in ec2_client.describe_instances()['Reservations']]
print(allInstances)
#<1#(39115337)~%(-1292132353)

In [ ]:
#>1#(39115337):(39115337)
#***Generated by: [AWS.Instance --list --name]
def getAllInstanceNames():
    instanceNames = []
    #[AWS.SearchInstancesFor]#@95753329 #<1#(39115337)~%(-822305474)
#>2#(95753329,39115337):(95753329)
    allInstances = ec2_client.describe_instances()
    for reservation in allInstances['Reservations']:
        for possibleInstance in reservation['Instances']:
#<2#(95753329)~%(1421253349) #>1#(39115337):(39115337)
            for tag in possibleInstance['Tags']:
                    if tag['Key'] == 'Name':
                        instanceNames.append(tag['Value'])
    return instanceNames
print('Function: getAllInstanceNames, compiled')
#<1#(39115337)~%(1217574603)

So, we wanted to be able just to see the names of the instances

In [ ]:
#[AWS.Instance --list --name]#@39115337 #>1#(39115337):(39115337)
instanceNames_1 = getAllInstanceNames()
print(instanceNames_1)
#<1#(18839326)~%(193362666)

In [ ]:
#>1#(18839326):(18839326)
#***Generated by: [AWS.Instance --list --state]
def getAllInstanceStates():
    instanceStates = []
    #[AWS.SearchInstancesFor]#@85184533 #<1#(18839326)~%(2044346738)
#>2#(85184533,18839326):(85184533)
    allInstances = ec2_client.describe_instances()
    for reservation in allInstances['Reservations']:
        for possibleInstance in reservation['Instances']:
#<2#(85184533)~%(1421253349) #>1#(18839326):(18839326)
            for tag in possibleInstance['Tags']:
                    if tag['Key'] == 'Name':
                        instanceStates.append((tag['Value'], possibleInstance['State']['Name']))
    return instanceStates
print('Function: getAllInstanceStates, compiled')
#<1#(18839326)~%(428971030)

Or, the running state of those instances

In [ ]:
#[AWS.Instance --list --state]#@18839326 #>1#(18839326):(18839326)
instanceStates_1 = getAllInstanceStates()
print(instanceStates_1)
#<1#(18839326)~%(-1575724888)

### Terminate Instances
Lastly, one of the biggest things is taking down instances you no longer need. Following the previous patterns, we wanted to do this easily

In [ ]:
#[AWS.Instance codespaceExample --terminate]#@20403255 #>1#(20403255):(20403255)
#[AWS.InstanceByName codespaceExample]#@54672827
instanceToterminate : any #<1#(20403255)~%(641053335) #>2#(54672827,20403255):(54672827)
#***Function: findInstanceByName, already created in scope
instanceToterminate = findInstanceByName("codespaceExample")
#<2#(54672827)~%(947861268) #>1#(20403255):(20403255)
response = ec2_client.terminate_instances(InstanceIds=[instanceToterminate['InstanceId']])
print(response)
#<1#(20403255)~%(-1593160191)

And that's it! We made this example for AWS scripting, but hopefully you can see how automations are pertinent to a wide range of scripts. And now, check out the next sandbox!